In [2]:
!pip install transformers einops accelerate bitsandbytes

#accelerate helps increase inference

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:551: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [8]:
checkpoint = "Falconsai/medical_summarization"
tokernizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map = "auto", torch_dtype=torch.float32)

In [10]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'Falconsai/medical_summarization',
    'device_map':'auto',
    'torch_dtype':'torch.float32'
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.c5.2xlarge",
	container_startup_health_check_timeout=300,
  )

# send request
predictor.predict({
	"inputs": "the need for magnetic resonance imaging ( mri ) in patients with an implanted pacemaker or implantable cardioverter - defibrillator ( icd ) is a growing clinical issue"})

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
-----------------*

UnexpectedStatusException: Error hosting endpoint huggingface-pytorch-tgi-inference-2023-12-04-20-11-36-458: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [ ]:

prompt = "the need for magnetic resonance imaging ( mri ) in patients with an implanted pacemaker or implantable cardioverter - defibrillator ( icd ) is a growing clinical issue . it is estimated that as many as 75% of active cardiac device recipients will become indicated for mri . currently , the vast majority of such devices are contraindicated for use with an mri . in european heart rhythm association survey , published recently for non - mri - certified icds ( 0.51.5 t field strength ) , the totally subcutaneous icd ( s - icd ) system , an implantable defibrillator with no leads that touch the heart , has recently been demonstrated to be a safe and effective defibrillator option for patients at risk for sudden cardiac death . it provides shock therapy and post - shock pacing therapy , but no long - term bradycardia pacing . although it has been shown as an alternative to the standard transvenous icd , its compatibility with mri remains unclear . various types of clinical mri systems currently use a superconductive magnet that creates a static magnetic field strength , typically 1.5 or 3 t. the use of mri with most pacemakers and icds is considered a contraindication due to potential hazards , including heating of the electrode that resides in or on the heart , damage to myocardium , elevation of pacing thresholds , unintended induction of ventricular tachycardia ( vt ) or ventricular fibrillation ( vf ) , pacing inhibition , permanent device malfunction , and distortion of the mri scan . recently , mr - conditional. mr - conditional indicates a lack of known hazards in a specified mri environment with specified conditions of use . due to the variety of mri scanners and scanning protocols , it is not practical to test even a single device under all conditions . hence , mr - conditional labelling dictates that the device is safe for use under certain scanning conditions , as well as how the cardiac device should be programmed before an exposure to the magnetic field in a mri scanner . the literature , although limited , provides some guidance for imaging patients with implanted pacemakers or icds that do not have mr - conditional labelling . this single - centre prospective non - controlled study describes the first use of mri in patients with an implanted s - icd . patients with implanted s - icd systems ( boston scientific sqrx model 1010 and q - trak model 3010 ) were enrolled for mri testing over a period of 18 months . the s - icd system implanted in this patient cohort was composed of a can implanted in a left mid - lateral pocket and a para - sternal subcutaneous electrode . the s - icd is currently not certified for use with an mri ; therefore , the ethics committee of homolka hospital , prague , czech republic approved our clinical study . patients with newly implanted s - icd systems ( < 6 weeks ) were excluded , and none of the patients had any intravascular leads . the patients were randomized for either a cardiac , brain , cervical , or lumbar spinal scan . one of the subjects underwent an additional knee examination , due to reported chronic pain . a total of 15 patients were enrolled into this study ( 12 males and three females , aged 2283 years , mean 53 years . subjects in our cohort ( table 1 ) underwent a total of 22 mri scans between 6 june 2012 and 24 december 2013 . in total , five brain scans , three cardiac scans , 12 lumbar scans , one knee , and one cervical spine scan were conducted ( table 2 ) . however , in one patient a minor disc protrusion was found , in other mri revealed stenosis of intervertebral foramen which was causing radicular pain of the nerve root l4 and based on this examination the patient was referred to ct - navigated periradicular therapy . table 1summary of patient anatomical data and scan locations , along with noted clinical eventsidagesexbmidgef , % indication for s - icdheating0164f20.5hcmp / vfs85secondary preventionnone0283m30.0post - mi / smvts post - catheter ablation/35secondary prevention ( post - transvenous icd extraction)none0331m25.3arvc / d / smvts68secondary preventionin - tolerable re - scanned0458m23.6post - mi / post - cabg30primary preventionnone0577m25.5post - mi30primary preventionnone0663m27.0post - mi30primary preventionnone0768m23.7post - mi / vfs / vts60secondary prevention post - transvenous icd extraction / svc occlusiontolerable0822m29.4brugada sy / vfs68secondary preventionin - tolerable re - scanned0959m27.1dcmp / vfs / post - mitral valve surgery/60secondary prev./post - transvenous icd extractionnone1041f24.6arvc / d70primary preventionnone1123f21.5lqts / vf60secondary preventionnone1266m36.9post - mi / vf / post - cabg50secondary prevention / post - repeat transvenous icd extractiontolerable1348m22.9dcmp(non - compaction)/vfs35secondary preventionnone1470m29systolic dysfunction of lv35primary preventionnone1526m33brugada sy65primary preventionnonehcmp , hypertrophic cardiomyopathy ; smvt , sustained monomorphic ventricular tachycardia ; mi , myocardial infarction ; arvc , arrhythmogenic right ventricular cardiomyopathy ; cabg , coronary artery by - pass graft ; lqts , long qt syndrom . table 2parmeters of s - icd and patient sensation during individual mri scansscan # idbody partheating sensationsshock zone ( b.p.m.)condit . shock zone ( b.p.m.)bat % episode num.101brainnone2302101001202brainnone240220861303l spinein - tolerable240220831403brainnone240220831504brainnone220190691605l spinenone220210541706l spinenone240220681807l spinetolerable240220582908l spinein - tolerablenananana1008brainnonenananana1108l spinenone2302108411209heartnone2402208911310l spinenone2301807911410heartnonenananana1511heartnone2301909711612l spinetolerable2001709721712l spinenone2001709421813c spinenone23019010041913l spinenone23019010042014l spinenone2301908612115kneenone25021010012215l spinenone2502101001s - icd parameters acquired prior- and post - mri were without any change , therefore only one value is presented.indices : na , not available ; l spine , lumbar spine ; c spine , cervical spine . summary of patient anatomical data and scan locations , along with noted clinical events hcmp , hypertrophic cardiomyopathy ; smvt , sustained monomorphic ventricular tachycardia ; mi , myocardial infarction ; arvc , arrhythmogenic right ventricular cardiomyopathy ; cabg , coronary artery by - pass graft ; lqts , long qt syndrom . parmeters of s - icd and patient sensation during individual mri scans s - icd parameters acquired prior- and post - mri were without any change , therefore only one value is presented . indices : na , not available ; l spine , lumbar spine ; c spine , cervical spine . studies were performed using a siemens avanto 1.5 t mri scanner ( vb17 software , quantum gradient coils ) . all scans were run in normal operating mode , which is limited to 2 w / kg whole body averaged specific absorption rate ( sar ) . clinically relevant mri sequences were used for evaluation ( see table 3 ) . table 3types of pulse sequences typically used for imaging of respective anatomical areasscan locationscan sequencesflairdwiflashfsehastesestirtruefispbrainxxxxheartxxxxcervical spinexxkneexxxxlumbar spinexxflair , fluid attenuated inversion recovery ; dwi , diffusion weighted imaging ; flash , fast low angle shot ; fse , fast spin echo ; haste , half acquisition single - shot turbo spin echo ; se , spin echo ; stir , short tau inversion recovery ; truefisp , true fast imaging with steady - state precession.fse sequence caused heating in subjects with a thermistor probe during lumbar spine examination ( see the text for details ) . types of pulse sequences typically used for imaging of respective anatomical areas flair , fluid attenuated inversion recovery ; dwi , diffusion weighted imaging ; flash , fast low angle shot ; fse , fast spin echo ; haste , half acquisition single - shot turbo spin echo ; se , spin echo ; stir , short tau inversion recovery ; truefisp , true fast imaging with steady - state precession . fse sequence caused heating in subjects with a thermistor probe during lumbar spine examination ( see the text for details ) . patients were asked to report immediately any pain , torqueing movement , or heating sensation in the area of the pocket or the electrode by pressing an emergency bulb . furthermore , all patients were questioned immediately following the mri procedure to ascertain any discomfort in the vicinity of the can or electrode . pulse oximetry and standard lead electrocardiogram ( ecg ) if discomfort occurred , the patient was asked if the scan could be repeated at a later time using a revised scan sequence or the subject was again randomized for another anatomical area . since none of the components of the s - icd system are on or in the heart , heating near or around however , heating near the electrode or can with the s - icd system may still cause serious patient discomfort . therefore , along with education of subjects , each patient was instrumented by taping an oesophageal temperature probe ( beta - therm model g22k7mcd8 ) on the skin over the mid - lateral implant site to record any temperature excursions that might be correlated to patient symptoms of heating / discomfort near the pocket . to minimize the risk of inappropriate therapy , the s - icd system was programmed to therapy each s - icd system was evaluated prior to and immediately after the scan to verify proper functioning , including interrogation , sensing , and battery voltage . after the completion of the mri , long - term regular clinical follow - up and checking of the device were performed . patients with implanted s - icd systems ( boston scientific sqrx model 1010 and q - trak model 3010 ) were enrolled for mri testing over a period of 18 months . the s - icd system implanted in this patient cohort was composed of a can implanted in a left mid - lateral pocket and a para - sternal subcutaneous electrode . the s - icd is currently not certified for use with an mri ; therefore , the ethics committee of homolka hospital , prague , czech republic approved our clinical study . patients with newly implanted s - icd systems ( < 6 weeks ) were excluded , and none of the patients had any intravascular leads . the patients were randomized for either a cardiac , brain , cervical , or lumbar spinal scan . one of the subjects underwent an additional knee examination , due to reported chronic pain . a total of 15 patients were enrolled into this study ( 12 males and three females , aged 2283 years , mean 53 years . subjects in our cohort ( table 1 ) underwent a total of 22 mri scans between 6 june 2012 and 24 december 2013 . in total , five brain scans , three cardiac scans , 12 lumbar scans , one knee , and one cervical spine scan were conducted ( table 2 ) . however , in one patient a minor disc protrusion was found , in other mri revealed stenosis of intervertebral foramen which was causing radicular pain of the nerve root l4 and based on this examination the patient was referred to ct - navigated periradicular therapy . table 1summary of patient anatomical data and scan locations , along with noted clinical eventsidagesexbmidgef , % indication for s - icdheating0164f20.5hcmp / vfs85secondary preventionnone0283m30.0post - mi / smvts post - catheter ablation/35secondary prevention ( post - transvenous icd extraction)none0331m25.3arvc / d / smvts68secondary preventionin - tolerable re - scanned0458m23.6post - mi / post - cabg30primary preventionnone0577m25.5post - mi30primary preventionnone0663m27.0post - mi30primary preventionnone0768m23.7post - mi / vfs / vts60secondary prevention post - transvenous icd extraction / svc occlusiontolerable0822m29.4brugada sy / vfs68secondary preventionin - tolerable re - scanned0959m27.1dcmp / vfs / post - mitral valve surgery/60secondary prev./post - transvenous icd extractionnone1041f24.6arvc / d70primary preventionnone1123f21.5lqts / vf60secondary preventionnone1266m36.9post - mi / vf / post - cabg50secondary prevention / post - repeat transvenous icd extractiontolerable1348m22.9dcmp(non - compaction)/vfs35secondary preventionnone1470m29systolic dysfunction of lv35primary preventionnone1526m33brugada sy65primary preventionnonehcmp , hypertrophic cardiomyopathy ; smvt , sustained monomorphic ventricular tachycardia ; mi , myocardial infarction ; arvc , arrhythmogenic right ventricular cardiomyopathy ; cabg , coronary artery by - pass graft ; lqts , long qt syndrom . table 2parmeters of s - icd and patient sensation during individual mri scansscan # idbody partheating sensationsshock zone ( b.p.m.)condit . shock zone ( b.p.m.)bat % episode num.101brainnone2302101001202brainnone240220861303l spinein - tolerable240220831403brainnone240220831504brainnone220190691605l spinenone220210541706l spinenone240220681807l spinetolerable240220582908l spinein - tolerablenananana1008brainnonenananana1108l spinenone2302108411209heartnone2402208911310l spinenone2301807911410heartnonenananana1511heartnone2301909711612l spinetolerable2001709721712l spinenone2001709421813c spinenone23019010041913l spinenone23019010042014l spinenone2301908612115kneenone25021010012215l spinenone2502101001s - icd parameters acquired prior- and post - mri were without any change , therefore only one value is presented.indices : na , not available ; l spine , lumbar spine ; c spine , cervical spine . summary of patient anatomical data and scan locations , along with noted clinical events hcmp , hypertrophic cardiomyopathy ; smvt , sustained monomorphic ventricular tachycardia ; mi , myocardial infarction ; arvc , arrhythmogenic right ventricular cardiomyopathy ; cabg , coronary artery by - pass graft ; lqts , long qt syndrom . parmeters of s - icd and patient sensation during individual mri scans s - icd parameters acquired prior- and post - mri were without any change , therefore only one value is presented . indices : na , not available ; l spine , lumbar spine ; c spine , cervical spine . studies were performed using a siemens avanto 1.5 t mri scanner ( vb17 software , quantum gradient coils ) . all scans were run in normal operating mode , which is limited to 2 w / kg whole body averaged specific absorption rate ( sar ) . clinically relevant mri sequences were used for evaluation ( see table 3 ) . table 3types of pulse sequences typically used for imaging of respective anatomical areasscan locationscan sequencesflairdwiflashfsehastesestirtruefispbrainxxxxheartxxxxcervical spinexxkneexxxxlumbar spinexxflair , fluid attenuated inversion recovery ; dwi , diffusion weighted imaging ; flash , fast low angle shot ; fse , fast spin echo ; haste , half acquisition single - shot turbo spin echo ; se , spin echo ; stir , short tau inversion recovery ; truefisp , true fast imaging with steady - state precession.fse sequence caused heating in subjects with a thermistor probe during lumbar spine examination ( see the text for details ) . types of pulse sequences typically used for imaging of respective anatomical areas flair , fluid attenuated inversion recovery ; dwi , diffusion weighted imaging ; flash , fast low angle shot ; fse , fast spin echo ; haste , half acquisition single - shot turbo spin echo ; se , spin echo ; stir , short tau inversion recovery ; truefisp , true fast imaging with steady - state precession . fse sequence caused heating in subjects with a thermistor probe during lumbar spine examination ( see the text for details ) . patients were asked to report immediately any pain , torqueing movement , or heating sensation in the area of the pocket or the electrode by pressing an emergency bulb . furthermore , all patients were questioned immediately following the mri procedure to ascertain any discomfort in the vicinity of the can or electrode . pulse oximetry and standard lead electrocardiogram ( ecg ) if discomfort occurred , the patient was asked if the scan could be repeated at a later time using a revised scan sequence or the subject was again randomized for another anatomical area . since none of the components of the s - icd system are on or in the heart , heating near or around the electrode can not harm the myocardium . however , heating near the electrode or can with the s - icd system may still cause serious patient discomfort . therefore , along with education of subjects , each patient was instrumented by taping an oesophageal temperature probe ( beta - therm model g22k7mcd8 ) on the skin over the mid - lateral implant site to record any temperature excursions that might be correlated to patient symptoms of heating / discomfort near the pocket . to minimize the risk of inappropriate therapy , the s - icd system was programmed to therapy each s - icd system was evaluated prior to and immediately after the scan to verify proper functioning , including interrogation , sensing , and battery voltage . after the completion of the mri , the s - icd system was reprogrammed to original settings . long - term regular clinical follow - up and checking of the device were performed . no anomalies were noted via pulse oximetry or ecg during the scans for any of the patients . eleven of 15 patients reported no sensation or pain from heating of the can , two of 15 patients reported feeling some heating , and two patients reported intolerable heating ( see table 2 ) . in patients with intolerable heating , the scan was halted within seconds and changed to a scan of the brain , which proceeded without incident . patient reports of heating in the vicinity of the can occurred only during lumbar scans with a thermistor probe ; no such reports occurred during scans of the brain , cardiac area , cervical spine , or without the probe . in two cases where heating in the vicinity of the can was reported by the patient , the scan sequence was altered to reduce the intensity of radiofrequency ( rf ) field exposure by reducing the turbo factor ( e.g. from 21 to 7 ) , increasing the repetition time ( e.g. to > 4000 ms ) , and reducing the flip angle ( e.g. from 170 to 120 ) . the target values were chosen arbitrarily to maintain image contrast ( flip angle ) and keep scan time at reasonable limits ( turbo factor and repetition time ) . less heating was noted by patients after these modifications to the scan parameters were made . 03 ) was observed to have a skin lesion , appearing to be a circular rash or ulcer on the surface of the skin over the can , approximately 35 mm in diameter . the cause of this skin anomaly is not known ; it was later noted to have fully healed at a follow - up 10 days after the scan . to ascertain the effect of heating due to the instrumented thermistor catheter , the two patients who experienced the heating ( examinations 9 and 16 , see table 2 ) were rescanned several weeks later without the thermistor catheter in place ( examinations 11 and 17 ) . first , modified sequence ( with even lower amount of energy deposited in the tissue ) was used , which caused no heating . as no sensation was reported by the subjects , they were asked to report even a minimal discomfort , and the lumbar scans were performed using the same settings that resulted in heating with the thermistor catheter in place in the first imaging session . the results of the rescans revealed that no heating was felt by the patients when the thermistor catheter was absent . there were no noted changes to battery voltage , ability to detect the qrs signal or stored diagnostic data . pacing thresholds can not be assessed by the s - icd system , so this was not evaluated . none of the patients reported any pulling or twisting of the can or pain from heating of the s - icd electrode . for scans of the brain , lumbar spine , knee , and cervical spine , no effect from image artefact was noted in the anatomical area of interest . however , for scans of the cardiac area , image artefact was noted to interfere with the ability to see parts of the left ventricle , though the right ventricle of the heart was unaffected and could be imaged usefully . this was due to the can and not the electrode ( see figure 1 ) , modifications to the protocol for the lumbar spine resulted in a lower signal - to - noise ratio ; however , the images remain in diagnostic quality ( see figure 2 ) . figure 1kinetic study in four - chamber view : the systolic ( a and c ) and diastolic ( b and d ) images of cine sequences , four - chamber view . the steady - state free precession ( ssfp ) sequence ( a and b ) shows more artefacts . in ssfp kinetic study , an inflow of dark blood from the left pulmonary veins was seen . it could be caused by s - icd but also by metallic ring in mitral annulus . the spoiled gradient echo ( gre ) sequence ( c and d ) is better , but an artefact at the lateral wall is obvious . figure 2lumbar spine imaging with icd : low sar t2 fse sequence ( upper image ) compared with normal t2 fse in the same subject ( lower image , for the scanning parameters see the discussion section ) . kinetic study in four - chamber view : the systolic ( a and c ) and diastolic ( b and d ) images of cine sequences , four - chamber view . the steady - state free precession ( ssfp ) sequence ( a and b ) shows more artefacts . in ssfp kinetic study , an inflow of dark blood from the left pulmonary veins was seen . it could be caused by s - icd but also by metallic ring in mitral annulus . the spoiled gradient echo ( gre ) sequence ( c and d ) is better , but an artefact at the lateral wall is obvious . lumbar spine imaging with icd : low sar t2 fse sequence ( upper image ) compared with normal t2 fse in the same subject ( lower image , for the scanning parameters see the discussion section ) . there were no noted changes to battery voltage , ability to detect the qrs signal or stored diagnostic data . pacing thresholds can not be assessed by the s - icd system , so this was not evaluated . none of the patients reported any pulling or twisting of the can or pain from heating of the s - icd electrode . for scans of the brain , lumbar spine , knee , and cervical spine , no effect from image artefact was noted in the anatomical area of interest . however , for scans of the cardiac area , image artefact was noted to interfere with the ability to see parts of the left ventricle , though the right ventricle of the heart was unaffected and could be imaged usefully . this was due to the can and not the electrode ( see figure 1 ) , modifications to the protocol for the lumbar spine resulted in a lower signal - to - noise ratio ; however , the images remain in diagnostic quality ( see figure 2 ) . figure 1kinetic study in four - chamber view : the systolic ( a and c ) and diastolic ( b and d ) images of cine sequences , four - chamber view . the steady - state free precession ( ssfp ) sequence ( a and b ) shows more artefacts . in ssfp kinetic study , an inflow of dark blood from the left pulmonary veins was seen . it could be caused by s - icd but also by metallic ring in mitral annulus . the spoiled gradient echo ( gre ) sequence ( c and d ) is better , but an artefact at the lateral wall is obvious . figure 2lumbar spine imaging with icd : low sar t2 fse sequence ( upper image ) compared with normal t2 fse in the same subject ( lower image , for the scanning parameters see the discussion section ) . kinetic study in four - chamber view : the systolic ( a and c ) and diastolic ( b and d ) images of cine sequences , four - chamber view . the steady - state free precession ( ssfp ) sequence ( a and b ) shows more artefacts . in ssfp kinetic study , an inflow of dark blood from the left pulmonary veins was seen . it could be caused by s - icd but also by metallic ring in mitral annulus . the spoiled gradient echo ( gre ) sequence ( c and d ) is better , but an artefact at the lateral wall is obvious . lumbar spine imaging with icd : low sar t2 fse sequence ( upper image ) compared with normal t2 fse in the same subject ( lower image , there are several reports in the current literature about mr - conditional pacemakers from several companies , but very limited reports about mr - conditional icds . biotronik announced in late 2011 release of their first mr - compatible icd device and defibrillator leads pro mri , but in the conditions of use excluded scanning of the torso and focused more on the extremities examination . in european heart rhythm association survey , 60% of centres did not implant any mri - certified icds , 34.3% implanted < 10 icd devices , and only 5.6% implanted 10 and more icds ; one - fifth of responders stated that mri - certified icds should be implanted in all patients but lack of reimbursement was indicated as a possible obstacle to implant more mri - certified pacemakers / icds by 47.1% of responding centres . none of the components of the s - icd system are on or in the heart . the s - icd depends less upon being in direct contact with the myocardium to function and instead uses far - field sensing and stimulation to provide the shock and post - shock pacing therapy . as a consequence , unlike transvenous systems heating near or around the electrode can not harm the myocardium , which could present with possible safety consequences such as an elevation in pacing thresholds or scarring of the myocardium , but it may still cause serious patient discomfort . because the s - icd is larger than modern transvenous icd 's , there may be more potential for the can to experience heating due to the magnetic gradient or rf field . we report results from what we believe is the first experience of mri scanning in patients with an implanted s - icd and in various anatomical areas . overall , mri was performed safely in all patients , which is in contrast to the current literature with mri imaging in patients with electrical - active devices which are not mri - conditional . in our study , the primary clinically significant event attributable to the mri scan was the occurrence of heating in the area of the pocket in the four patients that underwent lumbar scans . it was not known if this was due to the s - icd can itself or an artefact of the thermistor catheter used to measure skin temperature over the pocket . this required a revision of our protocol , which was to re - scan two of the patients who complained of heating . re - scanning of these patients without the thermistor probe resulted in no complaints of heating , so it is assumed that the thermistor catheter itself heated during the lumbar scans and caused the discomfort . as further evidence , all the heating complaints occurred during rf - intensive scan sequences ( namely fast spin echo ) with the temperature probe located axially near the centre of the bore , where rf fields are the highest . the thermistor catheter is constructed of insulated conductive cables connected to electrodes and should couple to the rf fields efficiently , causing heating at the electrodes and pain or damage on the surface of the skin where the probe was placed over the s - icd can . if the heating was due to the s - icd can itself , it would more likely occur during gradient - intensive scan sequences ( which can generate eddy currents on can surfaces and internal components ) and at locations in the bore where there are high gradient fields , such as near the bore edges . however , when the patient was scanned with gradient - intensive scan sequences ( e.g. flair dwi ) and with the s - icd system in high gradient field locations in the bore ( e.g. such as during a brain scan ) , patients did not detect any heating or discomfort . in addition , the subcutaneous lead , which was not instrumented with a thermistor catheter , never resulted in any heating sensation noted by the patient , even when exactly the same sequence that resulted in heating in the first session was used . the use of mri - compatible temperature monitors such as fibre optic temperature probes would have provided better confirmation of possible skin temperature elevation and would not have been affected by the rf fields . for cardiac imaging , the main problem to solve is metallic artefact , especially on the gradient - echo sequences . like in research performed by nazarian et al . , several scan protocols were used to see if any yielded different effects or reduced the qualitative extent of artefact . gradient mode was changed from normal to whisper , resulting in slower ramping of the field and therefore diminishing the changes of the magnetic field in time . artefacts when present were limited to blurring of the left ventricle during cardiac scans and most yielded clinically useful information . standard interrogation of the s - icd revealed no adverse effects upon the functioning of the system . while no adverse effects upon the post - scan s - icd device function were noted , not all possible scanning protocols were tested . it should be noted that , four of the s - icd 's were exposed to repeat mri scans without adverse effects to device function . in addition , because the s - icd does not provide long - term bradycardia pacing , it is assumed that pacemaker - dependent patients would not be implanted with this system . the inhibition of the pacemaker function during the scanning sequence and possible pacing threshold changes are a unique concern in patients implanted with transvenous icds . this study included only 15 patients and 22 scans done on the same 1.5 t mri scanner . thus , even these preliminary results should only be applied to 1.5 t mri scanners ( similarly as reported in the present literature for other implantable devices ) . device functionality was tested immediately after the scan but not for long - term effects . in addition , not all device functions were tested although the s - icd system does have a beeper / interrogation warning if battery levels or memory irregularities occur . however , patients were scheduled for regular check - up and no defect of the device was observed in following 725 months ( mean observation time 18 months ) . delayed enhancement mri for determining cardiac scarring was also not tested . also , there are other anatomical areas that were not evaluated , such as shoulder and knees . this study included only 15 patients and 22 scans done on the same 1.5 t mri scanner . thus , even these preliminary results should only be applied to 1.5 t mri scanners ( similarly as reported in the present literature for other implantable devices ) . device functionality was tested immediately after the scan but not for long - term effects . in addition , not all device functions were tested although the s - icd system does have a beeper / interrogation warning if battery levels or memory irregularities occur . however , patients were scheduled for regular check - up and no defect of the device was observed in following 725 months ( mean observation time 18 months ) . delayed enhancement mri for determining cardiac scarring was also not tested . also , there are other anatomical areas that were not evaluated , such as shoulder and knees . while more data are required to support a claim of mri - conditional , this study is the study to demonstrate the feasibility of exposing s - icd patients to mri using the scanning and monitor protocol described , with some precautionary measures including : ( i ) programming the device therapy off ; ( ii ) limiting the sar to 2.0 w / kg ; ( iii ) continuous monitoring of the patients pulse oximetry and ecg by qualified personnel and especially for any feelings of heating ; ( iv ) evaluate device function post scan ; ( v ) availability of full resuscitation facilities at the mri site . given the variables of different mri scanners , the decision to perform mri on patients with an implanted s - icd system should be balanced against the potential risks . in our study , the only heating was very likely introduced by not fully mri - compatible thermometer probe ; subjects rescanned without the probe did not report any abnormalities during the scan of any body area listed ( brain , cervical and lumbar spine , heart , and knee ) . this study was supported by iga mz r nt12094/2011 , research project charles university in prague , prvouk p34 and unce 204010/2012 . funding to pay the open access publication charges for this article was provided by iga mz r nt12094/2011 ."
payload = {
    "input":prompt,
    "parameters":{
        "do_sample":True,
        "top_p":0.7,
        "temperature":0.3,
        "top_k":50
    }
} 